In [1]:
%matplotlib inline 

import numpy as np
import pandas as pd
import pymysql as mdb
import sqlalchemy as sa
import matplotlib.pyplot as plt 
import re

import nltk
from nltk.corpus import stopwords

from gensim import corpora, models, similarities
import gensim.parsing.preprocessing as preprocess
import gensim.matutils as mat


In [2]:
# get user comments
con = mdb.connect(host='alfred-prod.cbyff1yl1xje.us-west-2.rds.amazonaws.com', 
                  user='insight', 
                  password='YWRt9fxYzx1PqHy', 
                  db='alfred_prod')
cur = con.cursor()
cur.execute('SHOW TABLES')
table_names = cur.fetchall()


def table_to_df(table):
    cur.execute('SELECT * from {table}'.format(table=table))
    rows = cur.fetchall()
    df = pd.DataFrame( [[ij for ij in i] for i in rows])
    cur.execute('DESCRIBE {table}'.format(table=table))
    cols = cur.fetchall()
    dfcols = [i[0] for i in cols]
    df.columns = dfcols
    return df


match_feedbacks=table_to_df('match_feedbacks')


In [3]:
stops = set(stopwords.words('english'))
stop_words = []
for s in stops:
    stop_words.append(s.encode('UTF-8'))
stops = set(stop_words)
stops.add("")
comments = []
for i in range(len(match_feedbacks)):
    #print i
    if match_feedbacks['reason'].iloc[i]:
        comments.append([word.lower() for word in re.split('\W+',match_feedbacks['reason'].iloc[i]) if word.lower() not in stops])
    

In [4]:
dictionary = corpora.Dictionary(comments)
corpus = [dictionary.doc2bow(text) for text in comments]

In [5]:
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, update_every=1, chunksize=10, passes=1,gamma_threshold=0.001, minimum_probability=0.005)

In [62]:
lda.print_topics(10)

[u'0.056*enjoyed + 0.054*time + 0.036*first + 0.031*best + 0.030*week + 0.028*things + 0.028*talking + 0.025*last + 0.024*cool + 0.021*even',
 u'0.045*looking + 0.034*information + 0.030*thanks + 0.028*exchange + 0.027*minutes + 0.026*wanted + 0.026*2 + 0.024*though + 0.023*also + 0.023*build',
 u'0.098*conversation + 0.051*able + 0.048*relevant + 0.041*much + 0.036*useful + 0.033*network + 0.029*us + 0.027*interesting + 0.024*anything + 0.024*wasn',
 u'0.159*meeting + 0.072*people + 0.032*weave + 0.031*experience + 0.031*would + 0.030*helpful + 0.026*learned + 0.024*company + 0.023*interested + 0.021*learn',
 u'0.074*match + 0.044*location + 0.035*professional + 0.033*common + 0.030*rating + 0.029*super + 0.023*met + 0.023*busy + 0.022*professionally + 0.020*etc',
 u'0.087*work + 0.075*get + 0.051*coffee + 0.038*connection + 0.029*believe + 0.026*concierge + 0.024*right + 0.017*made + 0.015*friends + 0.013*founder',
 u'0.086*meet + 0.064*fun + 0.051*like + 0.027*4 + 0.027*someone + 0.

In [63]:
lda.inference(corpus)

(array([[  3.09999896,   1.09999869,   0.1       , ...,   0.1       ,
           0.1       ,   0.1       ],
        [  0.10000866,  22.78578719,   0.10000779, ...,   0.10000564,
           0.10001104,   1.3950781 ],
        [  7.97713099,   0.10000904,   3.66108952, ...,   0.10000485,
           6.7760174 ,   0.10003839],
        ..., 
        [  3.89748411,   0.10000003,   1.53187145, ...,   3.78173061,
           0.10000772,   1.15901043],
        [  1.19971098,   2.43312673,   2.14880251, ...,   5.87193052,
           0.10000504,   1.13824099],
        [  0.1       ,   2.09999845,   3.10000165, ...,   0.1       ,
           0.1       ,   1.0999999 ]]), None)

In [6]:
import pyLDAvis.gensim
#data_created_using_prepare = 
data_created_using_prepare = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
data_created_using_prepare;

In [7]:
pyLDAvis.display(data_created_using_prepare)

# Topic Model on different ratings

### rating == 1

In [79]:
rate_star=1
comments = []
for i in range(len(match_feedbacks[match_feedbacks['rating']==rate_star])):
    #print i
    if match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]:
        comments.append([word.lower() for word in re.split('\W+',match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]) if word.lower() not in stops])

dictionary = corpora.Dictionary(comments)
corpus = [dictionary.doc2bow(text) for text in comments]
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, update_every=1, chunksize=10, passes=1,gamma_threshold=0.001, minimum_probability=0.005)
pyLDAvis.display(pyLDAvis.gensim.prepare(lda, corpus, dictionary))

### rating == 2

In [78]:

rate_star=2

comments = []

for i in range(len(match_feedbacks[match_feedbacks['rating']==rate_star])):

    #print i

    if match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]:
        comments.append([word.lower() for word in re.split('\W+',match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]) if word.lower() not in stops])


dictionary = corpora.Dictionary(comments)

corpus = [dictionary.doc2bow(text) for text in comments]

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, update_every=1, chunksize=10, passes=1,gamma_threshold=0.001, minimum_probability=0.005)

pyLDAvis.display(pyLDAvis.gensim.prepare(lda, corpus, dictionary))

### rating == 3

In [77]:

rate_star=3

comments = []

for i in range(len(match_feedbacks[match_feedbacks['rating']==rate_star])):

    #print i

    if match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]:
        comments.append([word.lower() for word in re.split('\W+',match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]) if word.lower() not in stops])


dictionary = corpora.Dictionary(comments)

corpus = [dictionary.doc2bow(text) for text in comments]

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, update_every=1, chunksize=10, passes=1,gamma_threshold=0.001, minimum_probability=0.005)

pyLDAvis.display(pyLDAvis.gensim.prepare(lda, corpus, dictionary))

### rating == 4

In [76]:
rate_star=4

comments = []

for i in range(len(match_feedbacks[match_feedbacks['rating']==rate_star])):

    #print i

    if match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]:
        comments.append([word.lower() for word in re.split('\W+',match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]) if word.lower() not in stops])


dictionary = corpora.Dictionary(comments)

corpus = [dictionary.doc2bow(text) for text in comments]

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, update_every=1, chunksize=10, passes=1,gamma_threshold=0.001, minimum_probability=0.005)

pyLDAvis.display(pyLDAvis.gensim.prepare(lda, corpus, dictionary))

### rating = 5

In [75]:
rate_star=5

comments = []

for i in range(len(match_feedbacks[match_feedbacks['rating']==rate_star])):

    #print i

    if match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]:
        comments.append([word.lower() for word in re.split('\W+',match_feedbacks[match_feedbacks['rating']==rate_star]['reason'].iloc[i]) if word.lower() not in stops])


dictionary = corpora.Dictionary(comments)

corpus = [dictionary.doc2bow(text) for text in comments]

lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=6, update_every=1, chunksize=10, passes=1,gamma_threshold=0.001, minimum_probability=0.005)

pyLDAvis.display(pyLDAvis.gensim.prepare(lda, corpus, dictionary))